In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-06-03"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
1022,2024-06-03,Argentina Liga A,21:00,Peñarol,San Martin,1.85,1.89,148.5,1.87,1.79,-1.5,1.98,2.03,2aFsnA0D,0.540541,0.529101,0.534759,0.558659,0.069641,0.0,0.4,NaN,NaN,NaN,NaN,NaN,141.028,42.109240,0.298588,0.000,163.300,112.20,112.32,0.0,0.0,0.0,0.0,0.015125,0.030912,0.017634,0.00,0.000,inf,0.0,0.0,0.0,1.06,0.212,4.198113,0.508816,0.5,-0.008816
1023,2024-06-03,Bélgica Liga Pro De Basquete,15:30,Antwerp Giants,Oostende,1.74,1.99,153.5,1.85,1.85,-2.5,2.01,2.40,QgldEIb3,0.574713,0.502513,0.540541,0.540541,0.077225,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,155.73,127.38,0.0,0.0,0.0,0.0,0.094786,0.000000,0.125067,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.000000,0.0,0.000000
1024,2024-06-03,Bolívia Libobasquet,21:30,Kinwa,Can Oruro,2.12,1.67,172.5,1.87,1.79,1.5,1.97,1.52,Ik0DhOWF,0.471698,0.598802,0.534759,0.558659,0.070501,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,167.44,0.00,0.0,0.0,0.0,0.0,0.167915,0.030912,0.182348,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.000000,0.0,0.000000
1025,2024-06-03,Bósnia E Herzegovina Prvenstvo Bih,13:00,Borac Banja Luka,Igokea,2.18,1.64,143.5,1.88,1.79,1.5,2.02,1.49,6axQh4Aa,0.458716,0.609756,0.531915,0.558659,0.068472,0.2,0.0,NaN,NaN,142.64,78.832179,0.552665,NaN,NaN,NaN,98.030,0.000,139.30,0.00,0.0,0.0,0.0,0.0,0.199915,0.034681,0.213542,1.67,0.334,3.532934,0.0,0.0,0.0,0.00,0.000,inf,0.000000,0.0,0.000000
1026,2024-06-03,França Pro B,15:45,La Rochelle,Boulazac,1.44,2.65,142.5,1.83,1.87,-6.5,2.04,4.20,4pP8NroK,0.694444,0.377358,0.546448,0.534759,0.071803,0.2,0.6,NaN,NaN,112.00,17.064095,0.152358,138.966,34.812286,0.250509,96.248,138.966,85.80,195.30,0.0,0.0,0.0,0.0,0.418386,0.015289,0.489535,1.27,0.254,1.732283,0.0,0.0,0.0,-1.08,-0.216,-7.638889,0.505110,0.4,-0.105110
1027,2024-06-03,Brasil Lbf Feminina,19:30,Corinthians F,Blumenau F,2.24,1.60,132.5,1.82,1.88,-2.5,1.87,1.83,YwQ8IhGc,0.446429,0.625000,0.549451,0.531915,0.071429,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,137.64,0.0,0.0,0.0,0.0,0.235702,0.022933,0.015289,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.000000,0.0,0.000000
1028,2024-06-03,Porto Rico Bsn,21:00,Osos de Manati,Gigantes de Carolina,1.65,2.15,188.5,1.84,1.84,-2.5,1.85,2.37,O2ooR4rT,0.606061,0.465116,0.543478,0.543478,0.071177,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,156.64,217.80,0.0,0.0,0.0,0.0,0.186081,0.000000,0.174263,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.000000,0.0,0.000000


## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,21:00,Argentina Liga A,Peñarol,San Martin,148.5,1.87,1.0,Over
1,15:30,Bélgica Liga Pro De Basquete,Antwerp Giants,Oostende,153.5,1.85,1.0,Over
2,15:45,França Pro B,La Rochelle,Boulazac,142.5,1.83,1.0,Over
3,21:00,Porto Rico Bsn,Osos de Manati,Gigantes de Carolina,188.5,1.84,1.0,Over
